# Data Preparation

In [3]:
import pandas as pd
import plotly.graph_objects as go

from data_preparation.additional_data_preparation import prepare_covid_dataset,get_mobility_data
from data_preparation.spatio_temporal_filtering import read_bd_data
from paths import *

In [4]:
metadata, series, metadata_region, region_series, metadata_country, country_series = read_bd_data()

In [5]:
time_series_bd = country_series

In [6]:
covid_cases,covid_period_start,covid_period_end = prepare_covid_dataset()

In [7]:
from mobility import bd_lockdown_dates

bd_lockdown_dates_df = pd.DataFrame(np.array(bd_lockdown_dates).reshape(-1,2),columns=['start_date','end_date'])
covid_cases['lockdown'] = 0
for idx,date in bd_lockdown_dates_df.iterrows():
    covid_cases.loc[date.start_date:date.end_date,'lockdown'] = 1

In [8]:
covid_cases

,confirmed,deaths,lockdown
2020-03-01,0.0,0.0,0
2020-03-02,0.0,0.0,0
2020-03-03,0.0,0.0,0
2020-03-04,0.0,0.0,0
2020-03-05,0.0,0.0,0
...,...,...,...
2022-02-24,1516.0,10.0,0
2022-02-25,1406.0,11.0,0
2022-02-26,759.0,8.0,0
2022-02-27,864.0,9.0,0


In [9]:
mobility = get_mobility_data()

In [10]:
mobility

,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
date,,,,,,
2020-02-15,4,6,10,7,8,1
2020-02-16,3,3,5,4,8,0
2020-02-17,1,4,3,2,5,1
2020-02-18,2,3,2,3,5,1
2020-02-19,1,1,3,2,5,1
...,...,...,...,...,...,...
2022-04-29,76,122,104,104,99,6
2022-04-30,68,116,126,106,37,7
2022-05-01,65,116,140,83,-16,12


In [11]:
covid_cases_two_year = covid_cases

mobility_two_year = mobility[covid_period_start:covid_period_end]
# mobility_two_year = mobility.shift(30)[covid_period_start:covid_period_end]

# Experiment

In [116]:
from statsmodels.tsa.seasonal import seasonal_decompose

# def yearly_seasonal_decomposition_bd(series):
#     # series_bd_weekly = series.resample('W', label='left', loffset=pd.DateOffset(days=1)).mean()
#     series_bd_weekly = series.resample('D', label='left', loffset=pd.DateOffset(days=0)).interpolate(method='linear')
#     # print(series_bd_weekly)
#     # .resample('W').mean().fillna(country_series.Bangladesh.mean())
#     result = seasonal_decompose(series_bd_weekly)
#     seasonal_component = result.seasonal.resample('H').ffill().reindex(series.index).ffill()
#     # stationary = (series - seasonal_component).to_frame()
#     stationary = (series - seasonal_component)
#     # stationary_diff = stationary.diff()
#     # PLotlyTimeSeries(stationary)
#     # PLotlyTimeSeries(stationary_diff)
#     return stationary

In [12]:
from exploration import yearly_seasonal_decomposition_bd

In [13]:
pm_two_year = time_series_bd[covid_period_start:covid_period_end]
pm_two_year_stationary = yearly_seasonal_decomposition_bd(pm_two_year.iloc[:,0])

pm_two_year_stationary

,0
time,
2020-03-01 00:00:00,60.017643
2020-03-01 01:00:00,62.037643
2020-03-01 02:00:00,67.757643
2020-03-01 03:00:00,34.617643
2020-03-01 04:00:00,70.297643
...,...
2022-02-28 19:00:00,91.447693
2022-02-28 20:00:00,88.077693
2022-02-28 21:00:00,49.777693


In [14]:
pm_two_year.iloc[:,0].resample('D', label='left', loffset=pd.DateOffset(days=1)).interpolate(method='linear')

time
2020-03-02     88.05
2020-03-03     89.21
2020-03-04     59.94
2020-03-05     50.34
2020-03-06     41.78
               ...  
2022-02-25     76.27
2022-02-26     52.52
2022-02-27     89.18
2022-02-28    115.14
2022-03-01    102.19
Freq: D, Name: Bangladesh, Length: 730, dtype: float64

In [20]:
from exploration import yearly_seasonal_decomposition_bd
from data_preparation.handling_missing_entries import FillMissingFromCombined, FillMissingFromNeighbors

time_series_bd_impute = time_series_bd.apply(FillMissingFromCombined).iloc[:,0]['2017-03-01':'2022-02-28']
time_series_bd_impute = time_series_bd_impute.resample('D').mean()
# time_series_bd_impute = FillMissingFromNeighbors(time_series_bd_impute)

time_series_stationary = yearly_seasonal_decomposition_bd(time_series_bd_impute)
# time_series_stationary = time_series_stationary.resample('D').mean()
# time_series_stationary = time_series_stationary[covid_period_start:covid_period_end]
time_series_stationary = time_series_stationary-time_series_stationary.min() # handle negative values
time_series_stationary

,0
time,
2017-03-01,120.545248
2017-03-02,89.995248
2017-03-03,74.809892
2017-03-04,62.885965
2017-03-05,48.421329
...,...
2022-02-24,64.594414
2022-02-25,63.773164
2022-02-26,77.011081


In [16]:
time_series_stationary.describe()
# (time_series_stationary-time_series_stationary.min()).describe()

,0
count,1826.000000
mean,73.857485
std,15.794410
min,0.000000
25%,65.766703
50%,73.792657
75%,82.439654
max,151.302186


In [ ]:
# Frequency and sampling rate
Fs = 262 # sampling rate
# Fs = 1826 # sampling rate
# Fs = 43824 # sampling rate
t = np.arange(0,1,1/Fs)
# Sine function
y = time_series_bd_impute.values
# Perform Fourier transform using scipy
from scipy import fftpack
y_fft = fftpack.fft(y)
# Plot data
n = np.size(t)
fr = Fs/2 * np.linspace(0,1,int(n/2))
y_m = 2/n * abs(y_fft[0:np.size(fr)])
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15,5))
ax[0].plot(t, y)    # plot time series
ax[1].stem(fr, y_m) # plot freq domain

In [ ]:
y_m
# sorted(y_m)[::-1]

In [139]:
time_series_bd_daily =  time_series_bd.resample('D').mean()

In [140]:
pm_two_year = pm_two_year.resample('D').mean()
# pm_two_year = time_series_stationary.resample('D').mean().iloc[:,0]
# pm_two_year = pm_two_year-pm_two_year.min()
# pm_two_year = pm_two_year.fillna(pm_two_year.median()).median(axis=1)
pm_two_year.name = 'pm reading'

In [141]:
pm_two_year

,Bangladesh
time,
2020-03-01,83.549167
2020-03-02,74.544167
2020-03-03,62.414167
2020-03-04,53.105417
2020-03-05,54.732083
...,...
2022-02-24,70.572917
2022-02-25,69.751667
2022-02-26,82.989583


In [142]:
from exploration import PLotlyTimeSeries

# PLotlyTimeSeries(pm_two_year.to_frame())
PLotlyTimeSeries(pm_two_year)

In [143]:
time_series_stationary

,0
time,
2017-03-01,120.545248
2017-03-02,89.995248
2017-03-03,74.809892
2017-03-04,62.885965
2017-03-05,48.421329
...,...
2022-02-24,64.594414
2022-02-25,63.773164
2022-02-26,77.011081


In [33]:
# PLotlyTimeSeries(time_series_stationary)
# PLotlyTimeSeries(time_series_stationary.resample('D').mean())

In [144]:
time_series_stationary

,0
time,
2017-03-01,120.545248
2017-03-02,89.995248
2017-03-03,74.809892
2017-03-04,62.885965
2017-03-05,48.421329
...,...
2022-02-24,64.594414
2022-02-25,63.773164
2022-02-26,77.011081


In [145]:
time_series_stationary_cp = time_series_stationary.copy()

In [146]:
low_mobility_days = mobility.index[(mobility.workplaces<0) & (mobility.transit_stations<0)]
# mobility_days = mobility.index[(mobility.workplaces<0) | (mobility.transit_stations<0) | (mobility.parks<0)]
low_mobility_days

DatetimeIndex(['2020-03-06', '2020-03-17', '2020-03-22', '2020-03-23',
               '2020-03-24', '2020-03-25', '2020-03-26', '2020-03-27',
               '2020-03-28', '2020-03-29',
               ...
               '2021-07-24', '2021-07-25', '2021-07-26', '2021-07-27',
               '2021-07-28', '2021-07-29', '2021-07-30', '2021-08-02',
               '2021-08-03', '2021-08-04'],
              dtype='datetime64[ns]', name='date', length=233, freq=None)

In [147]:
time_series_stationary_cp['low_mobility_days'] = False

In [160]:
for mobility_day in low_mobility_days:
    time_series_stationary_cp.loc[mobility_day,'low_mobility_days'] = True
time_series_stationary_cp.columns = ['PM2.5 Concentration (µgm<sup>-3</sup>)','low_mobility_days']

In [161]:
time_series_stationary_cp

,PM2.5 Concentration (µgm<sup>-3</sup>),low_mobility_days
time,,
2017-03-01,120.545248,False
2017-03-02,89.995248,False
2017-03-03,74.809892,False
2017-03-04,62.885965,False
2017-03-05,48.421329,False
...,...,...
2022-02-24,64.594414,False
2022-02-25,63.773164,False
2022-02-26,77.011081,False


In [163]:
# fig = px.scatter(time_series_stationary_cp,x=time_series_stationary.index,y='PM2.5 Concentration',color='low_mobility_days',color_discrete_map={True:'royalblue',False:'#E759AC'}) #rougepink
fig = px.scatter(time_series_stationary_cp,x=time_series_stationary.index,y='PM2.5 Concentration (µgm<sup>-3</sup>)',color='low_mobility_days',color_discrete_map={True:'#4D4D4F',False:'#F9F9F9'}) #rougepink
# fig = px.scatter(time_series_stationary_cp['2019-03':'2022-02'],x=time_series_stationary['2019-03':'2022-02'].index,y='PM2.5 Concentration',color='low_mobility_days',color_discrete_map={True:'royalblue',False:'#E759AC'}) #rougepink
fig.update_layout(
    autosize=False, width=1800, height=900,
    title="",
    xaxis_title="Time",
    font=dict(size=27, color="#3D3C3A"),
    legend_orientation='h',
    template='plotly_white'
)
fig.update_traces(marker=dict(line=dict(width=1, color='black')))
fig.show()

In [ ]:
time_series_stationary_cp

# Statistical Significance

In [ ]:
from scipy import stats

In [ ]:
group_a, group_b = time_series_stationary_cp.reading[time_series_stationary_cp.low_mobility_days], time_series_stationary_cp.reading[~time_series_stationary_cp.low_mobility_days]
t_value, p_value = stats.ttest_ind(group_a, group_b)
t_value, p_value

In [ ]:
time_series_stationary['2018-03':'2018-08'].mean(),time_series_stationary['2019-03':'2019-08'].mean(),time_series_stationary['2020-03':'2020-08'].mean(),time_series_stationary['2021-03':'2021-08'].mean()

In [ ]:
# regular = time_series_stationary['2017-04':'2020-03']
# regular = pd.concat((time_series_stationary['2017-03':'2017-08'],time_series_stationary['2018-03':'2018-08'],time_series_stationary['2019-03':'2019-08']))
regular = pd.concat((time_series_stationary['2017-03':'2017-11'],time_series_stationary['2018-03':'2018-11'],time_series_stationary['2019-03':'2019-11'],time_series_stationary['2021-03':'2021-11']))
# non_regular = time_series_stationary['2020-04':'2022-03']
# non_regular = pd.concat((time_series_stationary['2020-03':'2020-08'],time_series_stationary['2021-03':'2021-08']))
non_regular = time_series_stationary['2020-03':'2022-11']

In [ ]:
group_a, group_b = regular.dropna(), non_regular.dropna()

In [ ]:
group_a.mean(), group_b.mean()

In [ ]:
t_value, p_value = stats.ttest_ind(group_a, group_b)

t_value, p_value

# Correlation Analysis

In [164]:
mobility_covid_correlation = mobility_two_year.apply(lambda mobility_type : mobility_type.corr(covid_cases_two_year.lockdown))
mobility_covid_correlation

retail_and_recreation   -0.374650
grocery_and_pharmacy    -0.252039
parks                   -0.341118
transit_stations        -0.396158
workplaces              -0.375540
residential              0.599681
dtype: float64

In [165]:
mobility_pm_correlation = mobility_two_year.shift(0).apply(lambda mobility_type : mobility_type.corr(pm_two_year.Bangladesh))
# mobility_pm_correlation_shift_30 = mobility_two_year.shift(30).apply(lambda mobility_type : mobility_type.corr(pm_two_year))
mobility_pm_correlation

retail_and_recreation    0.387242
grocery_and_pharmacy     0.349897
parks                    0.396709
transit_stations         0.434335
workplaces               0.439753
residential             -0.392905
dtype: float64

In [21]:
time_series_stationary_for_mobility = time_series_stationary.iloc[:,0]
time_series_stationary_for_mobility['2020-03-01':'2022-02-28']

time
2020-03-01    77.570664
2020-03-02    71.716227
2020-03-03    59.586227
2020-03-04    50.277477
2020-03-05    51.904144
                ...    
2022-02-24    64.594414
2022-02-25    63.773164
2022-02-26    77.011081
2022-02-27    74.817748
2022-02-28    85.650811
Freq: D, Name: 0, Length: 730, dtype: float64

In [22]:
mobility_stationary_pm_correlation = mobility_two_year.shift(7).apply(lambda mobility_type : mobility_type.corr(time_series_stationary_for_mobility))
mobility_stationary_pm_correlation

retail_and_recreation   -0.125681
grocery_and_pharmacy    -0.103856
parks                   -0.147299
transit_stations        -0.123742
workplaces              -0.130375
residential              0.057379
dtype: float64

In [47]:
mobility_stationary_pm_correlation_list = []

for i in range(28):
    # mobility_stationary_pm_correlation = mobility_two_year.apply(lambda mobility_type : mobility_type.corr(time_series_stationary_for_mobility.shift(-i)))
    mobility_stationary_pm_correlation = mobility_two_year.apply(lambda mobility_type : mobility_type.corr(country_series.iloc[:,0].shift(-i)))
    mobility_stationary_pm_correlation_list.append(mobility_stationary_pm_correlation)

In [48]:
mobility_stationary_pm_correlation_df = pd.concat(mobility_stationary_pm_correlation_list,axis=1).T

In [51]:
mobility_stationary_pm_correlation_df

,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
0,0.356568,0.320778,0.372541,0.402715,0.401047,-0.363028
1,0.349200,0.314146,0.360272,0.395920,0.399184,-0.360586
2,0.354046,0.316988,0.362917,0.399724,0.402656,-0.363702
3,0.360654,0.328725,0.374382,0.407078,0.411444,-0.361343
4,0.364553,0.327319,0.377237,0.410361,0.420386,-0.379124
5,0.354686,0.317180,0.366373,0.401001,0.409926,-0.378523
6,0.341686,0.307790,0.349863,0.389229,0.400710,-0.371574
7,0.338396,0.305325,0.338922,0.382190,0.393096,-0.361016
8,0.342838,0.310648,0.339931,0.386002,0.400598,-0.360618
9,0.330698,0.299384,0.326386,0.372487,0.387737,-0.347065


In [50]:
mobility_stationary_pm_correlation_df.abs().idxmax()

retail_and_recreation    14
grocery_and_pharmacy     14
parks                    18
transit_stations         14
workplaces               14
residential              14
dtype: int64

In [56]:
mobility_stationary_pm_correlation_df.columns = ['Retail & <br> Recreation', 'Grocery & <br> Pharmacy', 'Parks',
       'Transit <br> Stations', 'Workplaces', 'Residential']

In [1]:
colors = [
    [0.0, '#3A3B3C'],  # Slate Gray for lowest values
    [0.5, '#FFFFFF'],  # White for middle values
    [1.0, '#2554C7']  # Royal Blue for highest values
]

fig = go.Figure(data=go.Heatmap(
    z=mobility_stationary_pm_correlation_df.T, y=mobility_stationary_pm_correlation_df.columns,
    x=mobility_stationary_pm_correlation_df.index,
    # colorscale='icefire',
    colorscale=colors,
    zmin=-.5, zmax=.5,
    reversescale=True,
    text=mobility_stationary_pm_correlation_df.T.round(2).values,  # Text labels for each cell
    # texttemplate="%{text}",
))

# colorscale='greys',

fig.update_layout(
    autosize=False, width=1860, height=620,
    title="",
    xaxis=dict(
        title="Lag Days",
        type="category",  # Set x-axis to cate+gorical
    ),
    yaxis_title="Mobility Type",
    font=dict(size=24, color="#3D3C3A")
)

fig.update_traces(
   colorbar=dict(
       title=dict(
           text="Correlation",
           side="right",
           font=dict(size=24)
       ),
       tickfont=dict(size=20),
       len=1
   )
)

fig.show(config={'displayModeBar': False, 'responsive': True})

NameError: name 'go' is not defined

In [ ]:
# pd.concat((time_series_stationary_for_mobility.shift(-7),mobility_two_year),axis=1).dropna().resample('W').mean().pct_change()
# pd.concat((time_series_stationary_for_mobility.shift(-7),mobility_two_year),axis=1).dropna()['2020-03':'2020-08'].diff().sum()
avg_2020 = pd.concat((time_series_stationary_for_mobility.shift(0),mobility_two_year),axis=1).dropna()['2020-03':'2020-11'].mean()
avg_2021 = pd.concat((time_series_stationary_for_mobility.shift(0),mobility_two_year),axis=1).dropna()['2021-03':'2021-11'].mean()

In [ ]:
time_series_stationary_for_mobility['2019-03':'2019-11'].mean()

In [ ]:
avg_2_year = pd.concat((avg_2021,avg_2020),axis=1)
avg_2_year.iloc[0] = (avg_2_year.iloc[0]-73.18)/73.18*100
avg_2_year.diff(axis=1)

In [ ]:
covid_cases_mobility_correlation = covid_cases_two_year.apply(lambda covid_cases_type : covid_cases_type.corr(mobility_two_year.residential))
covid_cases_mobility_correlation

In [ ]:
covid_cases_pm_correlation = covid_cases_two_year.shift(0).apply(lambda covid_cases_type : covid_cases_type.corr(pm_two_year))
covid_cases_pm_correlation_shift_30 = covid_cases_two_year.shift(30).apply(lambda covid_cases_type : covid_cases_type.corr(pm_two_year))
covid_cases_pm_correlation_shift_30

In [ ]:
# fig = px.line(mobility_two_year, x=mobility_two_year.index, y=mobility_two_year.columns)
fig = px.line(mobility_two_year.resample('W').mean(), x=mobility_two_year.resample('W').mean().index, y=mobility_two_year.columns)
fig.update_layout(legend_orientation='h')
fig.show()

In [ ]:
mobility_two_year_stacked = mobility_two_year.stack()
mobility_two_year_stacked.name = 'activity'
mobility_two_year_stacked = mobility_two_year_stacked.to_frame()
mobility_two_year_stacked['type'] = mobility_two_year_stacked.index.get_level_values(1)
mobility_two_year_stacked.index = mobility_two_year_stacked.index.get_level_values(0)

In [ ]:
mobility_two_year_stacked_pm = mobility_two_year_stacked.join(pm_two_year)
# mobility_two_year_stacked_pm

In [ ]:
df = mobility_two_year_stacked_pm
fig = px.density_contour(df, x="activity", y="pm reading", marginal_x="histogram", marginal_y="histogram",color='type')
fig.show()

In [ ]:
import plotly.express as px
fig = px.line(covid_cases, x=covid_cases.index, y=covid_cases.columns)
# fig = px.line(np.log(covid_cases+1), x=covid_cases.index, y=covid_cases.columns)
fig.update_layout(legend_orientation='h',font_size=15)
fig.show()

In [ ]:

fig = go.Figure()
fig.add_trace()

fig.show()

fig = go.Figure()
fig.add_trace()

fig.show()

In [51]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=3, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.02)
fig.add_trace(
    go.Scatter(x=covid_cases.index, y=covid_cases.confirmed, fill='tozeroy', mode='lines', line_color='indigo'),
    row=1, col=1)

fig.add_trace(
    go.Scatter(x=covid_cases.index, y=covid_cases.deaths, fill='tozeroy', mode='lines', line_color='red'),
    row=2, col=1)

fig.add_trace(
    go.Scatter(x=covid_cases.index, y=covid_cases.lockdown, fill='tozeroy', mode='lines', line_color='lightskyblue'),
    row=3, col=1)

fig.update_layout(height=600, width=1800,
                  title_text="Stacked Subplots with Shared X-Axes", legend_orientation='h',font_size=15)

fig.update_yaxes(title_text="confirmed cases", row=1, col=1)
fig.update_yaxes(title_text="death cases", row=2, col=1)
fig.update_yaxes(title_text="lockdown period", row=3, col=1)
fig.show()

In [ ]:
mobility_two_year.columns

In [50]:
fig = make_subplots(rows=6, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.02)

for i,mobility_type in enumerate(mobility_two_year[['retail_and_recreation', 'parks', 'grocery_and_pharmacy',  'workplaces','transit_stations',  'residential']]):
    fig.add_trace(
        # go.Scatter(x=mobility_two_year.index, y=mobility_two_year[mobility_type], fill='tozeroy', mode='lines', line_color='indigo'),
        go.Scatter(x=mobility_two_year.index, y=mobility_two_year[mobility_type], mode='lines'),
        row=i+1, col=1)
    fig.update_yaxes(title_text=mobility_type, row=i+1, col=1, range=[-100, 100],)

fig.update_layout(height=900, width=1800,
                  title_text="Stacked Subplots with Shared X-Axes", legend_orientation='h',font_size=12)

fig.show()

NameError: name 'make_subplots' is not defined

In [ ]:
covid_cases_two_year_stacked = np.log(covid_cases+1).stack()
covid_cases_two_year_stacked.name = 'case counts'
covid_cases_two_year_stacked = covid_cases_two_year_stacked.to_frame()
covid_cases_two_year_stacked['type'] = covid_cases_two_year_stacked.index.get_level_values(1)
covid_cases_two_year_stacked.index = covid_cases_two_year_stacked.index.get_level_values(0)

In [ ]:
# covid_cases_two_year_stacked_pm = covid_cases_two_year_stacked.join(np.log(pm_two_year+1))
covid_cases_two_year_stacked_pm = covid_cases_two_year_stacked.join(np.log(pm_two_year+100))
covid_cases_two_year_stacked_pm

In [ ]:
mobility_pm_correlation.name = 'mobility_pm_correlation'
mobility_pm_correlation_shift_30 = mobility_pm_correlation
mobility_pm_correlation_shift_30.name = 'mobility_pm_correlation_shift_30'
mobility_pm_correlations = pd.concat((mobility_pm_correlation,mobility_pm_correlation_shift_30),axis=1)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=mobility_pm_correlation.index,
    y=mobility_pm_correlation,
    name='mobility_pm_correlation',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=mobility_pm_correlation_shift_30.index,
    y=mobility_pm_correlation_shift_30,
    name='mobility_pm_correlation_shift_30',
    marker_color='lightsalmon'
))

fig.update_layout(barmode='group', xaxis_tickangle=-0, legend_orientation = 'h')
fig.show()

In [ ]:
df = covid_cases_two_year_stacked_pm
fig = px.density_contour(df, x="case counts", y="pm reading", marginal_x="histogram", marginal_y="histogram",color='type')
fig.show()

In [ ]:
import statsmodels.api as sm
pm_covid_mobility = pd.concat((pm_two_year.fillna(0),covid_cases_two_year,mobility_two_year),axis=1)
# pm_covid_mobility
model = sm.OLS(pm_covid_mobility['pm reading'], pm_covid_mobility.drop('pm reading', axis=1))
results = model.fit()
print(results.summary())

In [ ]:
mobility.to_csv('dhaka_mobility.csv')